# How to Concatenate OCO Lite Files Using Python

### Overview

This Jupyter Notebook demonstrates how to exploit remote OPeNDAP access and concatenate many [Orbiting Carbon Observatory-2 (OCO-2) Lite](https://disc.gsfc.nasa.gov/datasets/OCO2_L2_Lite_FP_11.1r/summary?keywords=OCO2_L2_Lite_FP) daily files into one data object, without downloading individual Lite files.

### Prerequisites

This notebook was written using Python 3.10, and requires these libraries and files:

- `netrc` file with valid Earthdata Login credentials
   - [How to Generate Earthdata Prerequisite Files](https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Generate%20Earthdata%20Prerequisite%20Files)
- [netCDF4-python](https://github.com/Unidata/netcdf4-python) (we recommend using version 1.6.2)
- [python-cmr](https://github.com/nasa/python_cmr)
- [NumPy](https://numpy.org/install/)

#### Optional Anaconda Environment YAML:

This notebook can be run using the ['opendap' YAML file](https://github.com/nasa/gesdisc-tutorials/tree/main/environments/opendap.yml) provided in the 'environments' subfolder.

Please follow the instructions [here](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file) to install and activate this environment. 

In [1]:
import os, sys
import datetime, time
import netCDF4 as nc
import numpy as np
from cmr import CollectionQuery, GranuleQuery, ToolQuery, ServiceQuery, VariableQuery

### 1. Set the time period to concatenate.

Note the ISO 8601 time format. It is critical to adhere to it.<br>
There are no checks to ensure the format is correct.

These time strings are used in the CMR query, and if the format is not correct, null<br>
result will be received.

In [2]:
t1 = datetime.datetime(2020,3,1)
t2 = datetime.datetime(2020,3,2,23,59)

###  2. Specify Earthdata dataset short name and dataset version strings.

Make sure to use the currently available dataset version.

In [3]:
short_name = 'OCO2_L2_Lite_FP'
vid = '11.1r'

### 3. Define function for retrieving OPeNDAP URLs

The function below will return a list of OPeNDAP URLs given a short name, version ID, start and end time, latitude/longitude points, or a bounding box.

In [4]:
def get_CMRgranurls(ShortName,VersionID,start_time,end_time,lon=None,lat=None,rad_km=None):
    """
    This program returns a list of OPeNDAP URLs.  If lon, lat, and radius are given, this function will search for all granules defined by that circle.  If lon and lat are 2 element lists, then this function will search for all granules defined by a bounding box.  If lon, lat, and radius_km are not given this function will search for all granules between the input start_time and end_time.  This function uses python-cmr to get the urls but it does not require prior knowledge of how many urls will be returned.
INPUTS
    "ShortName" the data set shortname
    "VersionID" the version ID of the product
    "start_time"  start time in utc.  The following is appended to the date: "T00:00:00Z.
    "end_time"  start time in utc The following is appended to the date: "T00:00:00Z, thus the end date is not actualy included in the results.
    "lon", "lat", "rad_km" (optional) will search within a radius or bounding box near a given location. If a radius or bounding box is not given, this function will return all granules within the input timerange.  
    OUTPUTS
    "urls" A list containing URLs
     
EXAMPLE
    Find all of the AIRX2RET granules within 30 km of  New Orleans
    > from opensearchtools.get_MOSurls import get_MOSurls
    > urls = get_CMRgranurls('AIRX2RET.006','2002.09.01','2016.01.01',lon=-90.0667,lat=29.95, rad_km=30.0)
    If a radius is not specified it will return all of the granules on the days in the search period.  For     example, following will return 241 granules.  The 1 extra if from 2002.08.31 part of which is on 2002.09.01
    > 
    > urls = get_CMRgranurls('AIRX2RET.006','2002.09.01','2002.09.02')
    HISTORY
    Created by Thomas Hearty,  March 14, 2016 
    """
    start_time = start_time.replace('.','-')
    end_time = end_time.replace('.','-')
    if len(start_time) == 10:
        start_time = start_time+"T00:00:00Z"
        end_time = end_time+"T00:00:00Z"

    # create a list of start and end times strings
    start_dt = datetime.datetime.fromtimestamp(time.mktime(time.strptime(start_time,"%Y-%m-%dT%H:%M:%SZ")))
    end_dt = datetime.datetime.fromtimestamp(time.mktime(time.strptime(end_time,"%Y-%m-%dT%H:%M:%SZ")))
    
    deltatime = end_dt - start_dt
    start_times = [start_time]
    end_times = []
    timeincrement = datetime.timedelta(7)
    while deltatime > timeincrement:
        old_start_dt = start_dt
        start_dt = old_start_dt+timeincrement
        end_times.append(start_dt.strftime("%Y-%m-%dT%H:%M:%SZ"))
        start_times.append(end_times[-1])
        deltatime = end_dt - start_dt
    end_times.append(end_time)
 
    opendap_urls = []
    for start_time_seg,end_time_seg in zip(start_times,end_times):
        api = GranuleQuery()

        # there are three types of searches that I can do.  1) global, 2) bounding box, 3) point radius
        if rad_km is None and lon is None and lat is None: # this is a global search
            granules = api.short_name(ShortName).version(VersionID).temporal(start_time_seg,end_time_seg).get(1000000) 
        elif rad_km is None and len(lon) == 2 and len(lat) == 2:
            granules = api.short_name(ShortName).version(VersionID).temporal(start_time_seg,end_time_seg).bounding_box(lon[0],lat[0],lon[1],lat[1]).get(1000000)
        else: # for now I will assume it is a circle
            granules = api.short_name(ShortName).version(VersionID).temporal(start_time_seg,end_time_seg).circle(lon,lat,rad_km*1000.).get(1000000)
        
        for granule in granules:
            for link in granule.get('links',[]):
                if 'rel' in link and 'href' in link and 'inherited' not in link:
                    if 'http://esipfed.org/ns/fedsearch/1.1/service#' in link['rel'] and 'opendap' in link['href']: # It's an opendap link
                        opendap_urls.append(link['href'])
            
    opendap_urls = list(set(opendap_urls)) # makes them unique and sorts them
    opendap_urls.sort() # this will probably put them in chronological order but it must be verified.

    return opendap_urls


Retrieve URLs and print them

In [5]:
urls = get_CMRgranurls(short_name,vid,t1.strftime('%Y-%m-%d'),t2.strftime('%Y.%m.%d'))
urls

['https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200229_B11100Ar_230603200059s.nc4',
 'https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200301_B11100Ar_230603200213s.nc4',
 'https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200302_B11100Ar_230603200315s.nc4']

This is only preparation of data objects in the name space.<br>
No storage is allocated here.

In [6]:
lon = []
lat = []
time = []
xco2 = []
qfsmpl = []
qcf = []

### 4. Load each granule and prepare for concatenation

In [7]:
for i in range(0, len(urls)):
    print(urls[i])
    fid = nc.Dataset(urls[i])
    xco20 = fid.variables['xco2'][:]
    lat0 = fid.variables['latitude'][:]
    lon0 = fid.variables['longitude'][:]
    qcf0 = fid.variables['xco2_quality_flag'][:]
    time0 = fid.variables['time'][:]

https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200229_B11100Ar_230603200059s.nc4
https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200301_B11100Ar_230603200213s.nc4
https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.11.1r/2020/oco2_LtCO2_200302_B11100Ar_230603200315s.nc4


### 5. Concatenate each variable

In [8]:
lon.append(lon0.filled())
lat.append(lat0.filled())
xco2.append(xco20.filled())
qcf.append(qcf0.filled())
time.append(time0.filled())

Below, all the variables have been concatenated as 1D vector variables in the computer memory.

An example quality screening, based on variable "xco2_quality_flag"<br>
from the Lite Full-Physics files. This quality flag is stored asvariable "qcf".<br>
The best quality is when qual=0.

In [9]:
xco2_all = np.hstack(xco2).squeeze()
lon_all  = np.hstack(lon).squeeze()
lat_all  = np.hstack(lat).squeeze()
qcf_all = np.hstack(qcf).squeeze()
time_all = np.hstack(time).squeeze()

Routine to subset best quality data points (qcf==0) out of all data points.

In [10]:
best = np.where(qcf_all==0)
xco2_best = xco2_all[best].squeeze()
lon_best = lon_all[best].squeeze()
lat_best = lat_all[best].squeeze()
time_best = time_all[best].squeeze()

### 6.  Save to a netCDF-4 file
It is simplified to the bare minimum to write into your current working directory.<br>
Make sure you have enough space.<br>
It will create or overwrite netCDF-4 data file "test.nc"

In [11]:
count = len(xco2_best)
foutid = nc.Dataset('test.nc',mode='w',format='NETCDF4_CLASSIC') 

In [12]:
dimtime = foutid.createDimension('time', None)
dimlat = foutid.createDimension('lat', count)
dimlon = foutid.createDimension('lon', count)
dimxco2 = foutid.createDimension('xco2', count)

In [13]:
varlon = foutid.createVariable('lon',float, ('lon',))
varlon.units = 'degrees_east'
varlon.long_name = 'longitude'

In [14]:
varlat = foutid.createVariable('lat',float, ('lat',))
varlat.units = 'degrees_north'
varlat.long_names = 'latitude'

In [15]:
vartime = foutid.createVariable('time',float, ('time',))
vartime.long_name = 'time'

In [16]:
varxco2 = foutid.createVariable('xco2',np.float64, ('xco2',))
varxco2.units = 'ppm'
varxco2.long_name = 'Bias-corrected, quality-filtered XCO2 on X2007 scale'

Fill the Variables

In [17]:
varlon[:] = lon_best
varlat[:] = lat_best
vartime[:] = time_best
varxco2[:] = xco2_best 

Display the contents of the dataset

In [18]:
foutid

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): time(78975), lat(78975), lon(78975), xco2(78975)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float64 xco2(xco2)
    groups: 

Close the dataset

In [19]:
foutid.close()